In [281]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

### Примеры функций потерь

In [283]:
def mse(x0, x):
    return (x0 - x) ** 2

def absolute(x0, x):
    return abs(x0 - x)

### Класс градиентного бустинга.
#### Используем деревья решений из sklearn

In [319]:
class MyGradientBoosting():
    def __init__(self, n_estimators = 100, learning_rate = 0.05, metric = None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.trees = []
        self.metric = self.__mse if metric is None else metric
        self.loss = []
        
    @staticmethod
    def __mse(x0, x):
        return (x0 - x) ** 2
    
    def __gradient(self, pred):
        dx = 1e-12
        return (self.metric(self.y, pred + dx) - self.metric(self.y, pred)) / dx
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
        r = self.y
        for i in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth = 3)
            tree.fit(X, r)
            pred = tree.predict(X)
            
            self.trees.append(tree)
            
            if not i:
                y_pred = pred
            else:
                y_pred += self.learning_rate * pred
                self.loss.append(self.metric(self.y, y_pred).sum())
            
            r = -self.__gradient(y_pred)
    
    def predict(self, X):
        pred = 0
        for i in range(self.n_estimators):
            pred += (self.learning_rate if i else 1) * self.trees[i].predict(X)
        return pred

### Создадим набор данных

In [293]:
X = np.random.randint(0, 10, (200, 2))
y = X[:, 0] ** 2 + X[:, 1] ** 2 + (np.random.rand(200, 1)*3).reshape(200)

### Создадим объект нашего класса и обучим на данных

In [320]:
mgb = MyGradientBoosting(learning_rate=0.05, metric=mse)
mgb.fit(X, y)
pred = mgb.predict(X)

print(f'Ошибка MGB: {mse(y, pred).sum()}')

Ошибка MGB: 843.7307609680506


### Создадим класс из sklearn и сравним

In [304]:
gbr = GradientBoostingRegressor(learning_rate=0.05)
gbr.fit(X, y)
pred = gbr.predict(X)

print(f'Ошибка GBR: {mse(y, pred).sum()}')

Ошибка GBR: 476.6345117176032


### <u>Результат</u>: среднеквадратичная ошибка для модели из sklearn ниже, что ожидаемо, так как наша модель проще по функциональности